In [ ]:
import vaex
import matplotlib
from vaex.ui.colormaps import cm_plusmin
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

##routine to split csv, SLOW: Run one time and then comment the line!

#df = vaex.from_csv(r'C:\Users\carlo\Dropbox\xampp2\mysql\data\business_navigator\companies-on-linkedin.csv', header=0, sep=';', convert=True, chunk_size=1_000_000)
df = vaex.open(r'companies-on-linkedin.csv.hdf5')

In [ ]:
#Cleaning, scaling down
df["Total_employee_estimate"] = df.Total_employee_estimate.astype('int')
df["Current_employee_estimate"] = df.Total_employee_estimate.astype('int')
df["Year_founded"] = df.Year_founded.astype('int')
df['lower_range'] = df.Size_range.str.slice(start=0, stop=1).astype('int')
df['upper_range'] = df.Size_range.str.slice(start=4, stop=6).astype('int')

def city (locality):
        try:
            return str(locality.split(',')[0])
        except:
            return 'None'
def estate (locality):
    try:
        return str(locality.split(',')[1])
    except:
        return 'None'
def country (locality):
    try:
        return str(locality.split(',')[2])
    except:
        return 'None'

df['city'] = df.Locality.apply(city)
df['estate'] = df.Locality.apply(estate)
df['country'] = df.Locality.apply(country)
df.info()

In [ ]:
#first descriptives
df = df[((df.Year_founded<=2021)) & ((df.Current_employee_estimate>=0))]
print(df.mean('Year_founded', binby=['Current_employee_estimate'], shape=4))

In [ ]:
print("sample size:", df.count(), "mean # of employees:", df.mean(df.Total_employee_estimate))
print("\n", df.groupby(by='Industry', agg={'mean_size': vaex.agg.mean('Current_employee_estimate'), 'max_size': vaex.agg.max('Current_employee_estimate')}))
print("\n", df.groupby(by='Industry').agg({'Industry': 'count','Total_employee_estimate': ['mean', 'std']}))

In [ ]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

years = [1900, 2013]
bins = 1000
counts_x = df.count(binby=df.Year_founded, limits=years, shape=bins)
counts_x

graph1 = plt.plot(np.linspace(years[0], years [1], bins), counts_x)
graph1.savefig('graph1.jpg')

### https://vaex.readthedocs.io/en/latest/tutorial.html

In [ ]:
companies_years = df.Year_founded.value_counts()
sns.barplot(x=companies_years.index, y=companies_years.values)